# Curve fitting

Linear least squares fitting can be used for fitting data to functions where the unknown parameters are linearly related to the output.  Here we will also look at other possibilities.

In [12]:
# Imports and settings
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
# Create a time base and a straight line fit
t = np.arange(0, 10, 0.01)
def stline(x, m, c):
    return m * x + c
y = stline(t, 3, 1.2)
# Add Gaussian noise
n = 1 * np.random.randn(len(t))
yn = y + n
plt.plot(t, y, t, yn)

In [ ]:
# Get errorbars from noise 
# plt.plot(t, y, t, yn)
# plt.errorbar(t[::5], yn[::5], np.std(n), fmt='ro')

# Least Squares Curve Fitting

Assume we know something about the function that underlies the observed data (for example, that it is linear or a polynomial function).  However, we don't know the coefficients of the various terms.  For example, say our function takes two parameters $p_1$ and $p_2$, and is a linear function of the time variable $t$: $g(t, p_1, p_2) = p_1 t + p_2$.  

We have a number of *observations* $g_1, g_2, \ldots, g_n$ of this function at different time instants $t_1, t_2, \ldots, t_n$.  These observations can then be written as:

$$
\mathbf{g} \equiv
\begin{pmatrix}
g_1 \\
g_2 \\
\vdots \\
g_n
\end{pmatrix}
=
\begin{pmatrix}
t_1 & 1 \\
t_2 & 1 \\
\vdots & \vdots \\
t_n & 1
\end{pmatrix}
\begin{pmatrix}
p_1 \\
p_2
\end{pmatrix}
\equiv
\mathbf{M}\mathbf{p}
$$

## Mean Square Error
We can therefore define an error $\varepsilon = \mathbf{Mp}-\mathbf{g}$ (note that this is itself a vector of point-wise errors), and a *mean-square error* or MSE as:

$$
E = \varepsilon^T \varepsilon = \sum_{1}^{N} \varepsilon_i = \sum_1^N ((p_1 t_i + p_2) - g_i)^2
$$

The goal of *least squares fitting* is to find the parameters $p_i$ such that this MSE $E$ is minimized.  More details of how this works can be seen at [LibreTexts](https://math.libretexts.org/Bookshelves/Linear_Algebra/Interactive_Linear_Algebra_(Margalit_and_Rabinoff)/06%3A_Orthogonality/6.5%3A_The_Method_of_Least_Squares).

In our case, we can use the `lstsq` function from the `numpy.linalg` library.  For this, we have to construct the $\mathbf{M}$ matrix.

In [ ]:
# Use column_stack to put the vectors side by side
M = np.column_stack([t, np.ones(len(t))])
# Use the lstsq function to solve for p_1 and p_2
(p1, p2), _, _, _ = np.linalg.lstsq(M, yn, rcond=None)
print(f"The estimated equation is {p1} t + {p2}")

In [ ]:
# Plot against the original input and compare
yest = stline(t, p1, p2)
plt.plot(t, y, t, yn, t, yest)

## More complicated functions

Note that we only require the relation with the parameters to be linear.  The function itself can be nonlinear.

In [ ]:
# Polynomial fit
def f1(x, p):
    return p[0] * np.sin(x) + p[1] * np.sin(3*x) + p[2]

t = np.arange(-3, 3, 0.01)
p = (2.0, 3.5, 1.0)
y = f1(t, p)

yn = y + np.random.randn(len(y))
# plt.plot(t, y, t, yn)

In [ ]:
# Build a model
# Use column_stack to put the vectors side by side
# We use the same time base `t` since we have yn (measurements) for this time base
M = np.column_stack([np.sin(t), np.sin(3*t), np.ones(len(t))])
# Use the lstsq function to solve for p_1 and p_2
p, _, _, _ = np.linalg.lstsq(M, yn, rcond=None)
print(f"The estimated parameters are: {p[0]} sin(t) + {p[1]} sin(3t) + {p[2]}")

# Non-linear curve fitting

What if your equation was not a linear function of the parameters?  For example:
$$g(t; p_1, p_2) = e^{-p_1 t} + p_2$$

The problem here is that we cannot create the $M$ matrix as a linear combination of $p_1$ and $p_2$!  We still have a notion of MSE:

$$E = \sum_1^N (g(t; p_1, p_2) - z_t)^2$$
where $z_t$ are the observed values.  However, the least squares minimization techniques discussed earlier do not work.  

## `curve_fit`

The `scipy.optimize` library contains the `curve_fit` function that can perform a non-linear curve fitting on observed data.  Unlike the least squares method, here we need to feed in a parametrized function that can be used to estimate the parameters.  

In [21]:
# Create a function with nonlinear dependence on parameters
def nlfunc(t, p1, p2):
    return np.exp(-p1 * t) + p2
t = np.arange(0, 10, 0.01)
z = nlfunc(t, 0.5, 0.5)
# Reuse the same noise - we are lazy
n = np.random.randn(len(z))
zn = z + 0.2 * n
plt.plot(t, z, t, zn)
plt.errorbar(t[::5], zn[::5], np.std(n), fmt='ro')

<ErrorbarContainer object of 3 artists>

In [22]:
# Set up the non-linear curve fit
from scipy.optimize import curve_fit
(zp1, zp2), pcov = curve_fit(nlfunc, t, zn)
print(f"Estimated function: exp(-{zp1}t) + {zp2}")

Estimated function: exp(-1.0t) + -0.3003061564786509


/Users/nitin/miniforge3/envs/appdev/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [ ]:
zest = nlfunc(t, zp1, zp2)
plt.plot(t, z, t, zest)

In [ ]:
# Non-linear sinusoidal function
def sinfunc(t, p1, p2):
    return p1 * np.sin(2 * np.pi * p2 * t)
s = sinfunc(t, 5, 0.5)
sn = s + 0.2*n
# Fit with only first K points
K = len(s)
(sp1, sp2), _ = curve_fit(sinfunc, t[:K], sn[:K])
print(f"Estimated over {K} out of {len(s)} samples: {sp1} * sin(2*pi*{sp2}*t)")
# Regenerate data
sest = sinfunc(t, sp1, sp2)
plt.plot(t, s, t, sest)